# 03 — Full TitansMemoryLayer: Kết Hợp Tất Cả

`TitansMemoryLayer` kết hợp **Surprise Metric** + **Memory Module** thành pipeline hoàn chỉnh:

```
Mỗi timestep:
  1. Tính surprise (dữ liệu này có bất ngờ không?)
  2. Đọc bộ nhớ (memory dự đoán gì?)
  3. Ghi bộ nhớ gated by surprise (ghi nhớ nếu mới lạ)
  4. Cập nhật surprise predictor (học pattern)
  5. Áp dụng forgetting (quên thông tin cũ)
```

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from titans_memory import TitansMemoryLayer, generate_repeating_with_anomalies

torch.manual_seed(42)

# Tạo chuỗi với pattern [1,2,3,4,5] lặp lại, có anomalies
seq, mask = generate_repeating_with_anomalies(
    pattern=[1.0, 2.0, 3.0, 4.0, 5.0],
    repeats=12,
    anomaly_indices=[12, 30, 48],
    anomaly_value=50.0,
)

layer = TitansMemoryLayer(
    input_dim=1, memory_dim=16, hidden_dim=16,
    momentum=0.9, forget_rate=0.01,
)
result = layer.process_sequence(seq.unsqueeze(-1))

print(f"Chuỗi: {len(seq)} phần tử, {int(mask.sum())} anomalies")
print(f"Keys trong result: {list(result.keys())}")
print(f"Số surprise scores: {len(result['surprise_scores'])}")

In [ ]:
scores = [s.item() for s in result["surprise_scores"]]
momentum = [s.item() for s in result["surprise_momentum"]]
anomaly_idx = mask.nonzero().squeeze(-1).numpy()

fig, axes = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

# Input sequence
axes[0].plot(seq.numpy(), color="#2196F3", linewidth=1)
axes[0].scatter(anomaly_idx, seq[mask.bool()].numpy(), color="#F44336", s=80, zorder=5)
axes[0].set_ylabel("Giá trị")
axes[0].set_title("Input Sequence (đỏ = anomalies)")

# Raw surprise
axes[1].plot(scores, color="#FF9800", linewidth=1.5)
for idx in anomaly_idx:
    axes[1].axvline(x=idx, color="#F44336", alpha=0.3, linestyle="--")
axes[1].set_ylabel("Surprise")
axes[1].set_title("Raw Surprise Score")

# Momentum surprise
axes[2].plot(momentum, color="#E91E63", linewidth=2)
for idx in anomaly_idx:
    axes[2].axvline(x=idx, color="#F44336", alpha=0.3, linestyle="--")
axes[2].set_xlabel("Timestep")
axes[2].set_ylabel("Surprise")
axes[2].set_title("Momentum-Smoothed Surprise (beta=0.9)")

plt.tight_layout()
plt.show()

## So sánh Hyperparameters

Thay đổi `momentum` và `forget_rate` ảnh hưởng lớn đến hành vi memory:

- **Momentum cao** (0.9): surprise giảm chậm sau anomaly → memory "cảnh giác" lâu hơn
- **Momentum thấp** (0.1): surprise giảm nhanh → phản ứng nhạy nhưng quên nhanh
- **Forget rate cao**: quên nhanh → chỉ nhớ gần đây
- **Forget rate thấp**: quên chậm → nhớ lâu hơn

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# So sánh momentum
for beta in [0.0, 0.5, 0.9, 0.99]:
    torch.manual_seed(42)
    layer = TitansMemoryLayer(
        input_dim=1, memory_dim=16, hidden_dim=16, momentum=beta, forget_rate=0.01
    )
    result = layer.process_sequence(seq.unsqueeze(-1))
    m_scores = [s.item() for s in result["surprise_momentum"]]
    axes[0].plot(m_scores, linewidth=1.5, label=f"momentum={beta}")

axes[0].set_title("Ảnh hưởng của Momentum")
axes[0].set_xlabel("Timestep")
axes[0].set_ylabel("Momentum Surprise")
axes[0].legend(fontsize=9)

# So sánh forget rate
for rate in [0.0, 0.01, 0.05, 0.1]:
    torch.manual_seed(42)
    layer = TitansMemoryLayer(
        input_dim=1, memory_dim=16, hidden_dim=16, momentum=0.9, forget_rate=rate
    )
    result = layer.process_sequence(seq.unsqueeze(-1))
    norms = [sum(w.norm().item() for w in s) for s in result["memory_snapshots"]]
    axes[1].plot(norms, linewidth=1.5, label=f"forget_rate={rate}")

axes[1].set_title("Ảnh hưởng của Forget Rate")
axes[1].set_xlabel("Timestep")
axes[1].set_ylabel("Memory Weight Norm")
axes[1].legend(fontsize=9)

plt.tight_layout()
plt.show()

## Kết luận

- TitansMemoryLayer xử lý chuỗi **từng bước**, quyết định ghi nhớ dựa trên surprise
- **Momentum** giúp "làm mịn" surprise, tránh phản ứng thái quá với nhiễu
- **Forget rate** kiểm soát tốc độ quên — cân bằng giữa nhớ lâu và cập nhật mới
- Anomalies luôn gây surprise cao → được ghi nhớ mạnh → đây là behavior mong muốn!